## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Your name here: Marissa Green
## Pair Programming Group Number: 22

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [18]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [29]:
df_X, s_y = load_diabetes(return_X_y = True, as_frame = True)

# for linear regression
intercept = pd.DataFrame({"intercept":np.ones(len(df_X))})
df_X = pd.concat([intercept, df_X], axis = 1)

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

In [32]:
def get_linear_regression_model( df_X, s_y ):
    # your code here   
    # find beta hat per the formula (3.6) (you should use the library we used in class)
    [beta_hat, residuals, rank, s] = np.linalg.lstsq(df_X, s_y, rcond=-1)
    
    return beta_hat.reshape(len(df_X.columns), 1)

In [33]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model(df_X, s_y)
beta_hat

array([[ 152.13348416],
       [ -10.01219782],
       [-239.81908937],
       [ 519.83978679],
       [ 324.39042769],
       [-792.18416163],
       [ 476.74583782],
       [ 101.04457032],
       [ 177.06417623],
       [ 751.27932109],
       [  67.62538639]])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [34]:
def partition_data( df_X, s_y, k ):
    # your code here
    
    # add in random_ints to randomly partition data
    random_ints = pd.DataFrame(np.random.randint(1, 6, len(df_X)))
    df_X_with_random = pd.concat([random_ints, df_X], axis=1)
    s_y_with_random = pd.concat([random_ints, s_y], axis=1)

    dict_k_df_X = {}
    dict_k_s_y = {}
    
    for i in range(1,k+1):
        dict_k_df_X[i] = df_X[df_X_with_random[0] == i]
        dict_k_s_y[i] = s_y[s_y_with_random[0] == i]
        
    return dict_k_df_X, dict_k_s_y

In [35]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [36]:
# Check fold sizes
sum_len_df = 0
sum_len_series = 0

for i in range(1,6):
    len_df = len(dict_k_df_X[i])
    len_series = len(dict_k_s_y[i]) 
    sum_len_df += len_df
    sum_len_series += len_series
    print("Fold", i, "length of dataframe is", len_df ,"and length of series is", len_series)

print("The sum of the number of elements in each fold is",sum_len_df, sum_len_series, "and there are", len(df_X),"rows in the original df")

Fold 1 length of dataframe is 97 and length of series is 97
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 73 and length of series is 73
Fold 4 length of dataframe is 82 and length of series is 82
Fold 5 length of dataframe is 94 and length of series is 94
The sum of the number of elements in each fold is 442 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [37]:
def get_mae( s_y, s_y_hat):
    # your code here
    mae = 0
    
    # BETTER WAY: mae = np.absolute(s_y - s_y_hat)
    for i in range(len(s_y)):
        mae += np.absolute(s_y[i] - s_y_hat[i])/len(s_y)
    
    return mae   

In [38]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [39]:
mae = np.array([])
for k in dict_k_df_X.keys():
    # your code here
    
    # wrong, need s_y and df_X to be all folds NOT k (opposite of just k)
    # CORRECTED, NEED TO CALCULATE BETA_HAT PER EACH FOLD
    s_y = dict_k_s_y[k].to_numpy()
    # X = dict_k_df_X[k]
    # beta_hat = get_linear_regression_model(X, s_y)
    
    s_y_hat = np.dot(dict_k_df_X[k], beta_hat)
    
    mae = np.append( mae, get_mae(s_y,s_y_hat) )  

In [40]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 37.01, the max MAE is 48.67, and the mean MAE is 43.27


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [41]:
df_X, s_y = load_iris(return_X_y = True, as_frame = True)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [42]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )
# Check fold sizes
sum_len_df = 0
sum_len_series = 0
for i in range(1,6):
    len_df = len(dict_k_df_X[i])
    len_series = len(dict_k_s_y[i]) 
    sum_len_df += len_df
    sum_len_series += len_series
    print("Fold", i, "length of dataframe is", len_df ,"and length of series is", len_series)
print("The sum of the number of elements in each fold is",sum_len_df, sum_len_series, "and there are", len(df_X),"rows in the original df")

Fold 1 length of dataframe is 27 and length of series is 27
Fold 2 length of dataframe is 39 and length of series is 39
Fold 3 length of dataframe is 30 and length of series is 30
Fold 4 length of dataframe is 18 and length of series is 18
Fold 5 length of dataframe is 36 and length of series is 36
The sum of the number of elements in each fold is 150 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [43]:
def get_acc( s_1, s_2 ):
    # your code here
    equal = 0
    for i in range(len(s_1)):
        if s_1[i] == s_2[i]:
            equal += 1
    
    return equal/len(s_1)

In [44]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [45]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4]) 
# no over training happening because overtraining/overfitting would happen at smaller impurities.
# This actually underfit data

# Outer loop
outer_acc = np.array([])

for k in dict_k_df_X.keys():

    # your code here, partition of 1 and 4
    outer_test_df = dict_k_df_X[k]
    outer_test_s_y = dict_k_s_y[k]
    outer_train_df = pd.DataFrame()
    outer_train_s_y = pd.DataFrame()
    for i in dict_k_df_X.keys() - [k]:
        outer_train_df = pd.concat([outer_train_df, dict_k_df_X[i]], axis=0)
        outer_train_s_y = pd.concat([outer_train_s_y, dict_k_s_y[i]], axis=0)

    highest_acc = 0
    
    for pos_min_impurity in possible_min_impurity_decrease:
        print("Testing", pos_min_impurity, "min impurity decrease")
        
        inner_acc = np.array([])
        
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k
        for i in dict_k_df_X.keys() - [k]:
            
            # set up partitions for inner loop
            inner_test_df = dict_k_df_X[i]
            inner_test_s_y = dict_k_s_y[i]
            inner_train_df = pd.DataFrame()
            inner_train_s_y = pd.DataFrame()
            
            # CORRECTED, FORGOT TO SUBTRACT BOTH I AND K :(
            for j in dict_k_df_X.keys() - [i,k]:
                inner_train_df = pd.concat([inner_train_df, dict_k_df_X[j]], axis=0)
                inner_train_s_y = pd.concat([inner_train_s_y, dict_k_s_y[j]], axis=0)
            
            # making a tree for each fold
            decision_tree = tree.DecisionTreeClassifier(criterion="gini", min_impurity_decrease = pos_min_impurity)
            decision_tree = decision_tree.fit(inner_train_df, inner_train_s_y)
            
            prediction = decision_tree.predict(inner_test_df)
            
            # calculating the accuracy for this fold
            this_acc_1 = get_acc( prediction, np.array(inner_test_s_y))
            inner_acc = np.append(inner_acc, this_acc_1)
        
        print("    Average accuracy over 4 folds is", np.round(np.mean(inner_acc), 3))
            
    # Use best min impurity decrease to train model
    highest_acc_indx = inner_acc.argmax(axis=0)
    highest_acc_val = possible_min_impurity_decrease[highest_acc_indx]
    print("\nBest min impurity decrease is", highest_acc_val, "\n\n")
    
    decision_tree = tree.DecisionTreeClassifier(criterion="gini", min_impurity_decrease = highest_acc_val)
    decision_tree = decision_tree.fit(outer_train_df, outer_train_s_y)
    
    # outer accuracy calculation 
    prediction = decision_tree.predict(outer_test_df)
    this_acc = get_acc( prediction, np.array(outer_test_s_y))
    outer_acc = np.append(outer_acc, this_acc) # make sure and calculate this_acc in your loop    

Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.916
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.819
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.627
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.251

Best min impurity decrease is 0.4 


Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.932
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.932
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.775
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.376

Best min impurity decrease is 0.3 


Testing 0.1 min impurity decrease
    Average accuracy over 4 folds is 0.966
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.966
Testing 0.3 min impurity decrease
    Average accuracy over 4 folds is 0.686
Testing 0.4 min impurity decrease
    Average accuracy over 4 folds is 0.2

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [46]:
print("My minimum accuracy of the outer fold test sets is", np.round(outer_acc.min(), 3))
print("My maximum accuracy of the outer fold test sets is", np.round(outer_acc.max(), 3))
print("My avergae accuracy of the outer fold test sets is", np.round(np.mean(outer_acc), 3))

My minimum accuracy of the outer fold test sets is 0.296
My maximum accuracy of the outer fold test sets is 0.933
My avergae accuracy of the outer fold test sets is 0.607
